In [1]:
import pandas as pd
import datetime as dt
import numpy as np

In [2]:
from oandapyV20 import API
from oandapyV20.contrib.factories import InstrumentsCandlesFactory
import oandapyV20.endpoints.forexlabs as labs
import oandapyV20.endpoints.orders as orders
import oandapyV20.endpoints.trades as trades

## Get sample data

In [3]:
def hist(api, instrument, start_days, end_days, granularity):

    start_date = (dt.datetime.now()-dt.timedelta(days=start_days)).strftime('%Y-%m-%dT%H:%M:%SZ')
    end_date = (dt.datetime.now()-dt.timedelta(days=end_days, hours=2,minutes=4)).strftime('%Y-%m-%dT%H:%M:%SZ')

    params ={
                "from": start_date,
                "to": end_date,
                "granularity":granularity,
            }

    df_list = []
    for r in InstrumentsCandlesFactory(instrument=instrument,params=params):
        api.request(r)
        df = pd.DataFrame(r.response['candles'])
        if(df.empty==False):
            time = df['time']
            volume = pd.DataFrame(df['volume'].apply(pd.Series))
            df = pd.DataFrame(df['mid'].apply(pd.Series))
            df = pd.concat([df,time,volume], axis=1)
            df['time'] = pd.to_datetime(df['time'], format='%Y-%m-%dT%H:%M:%S.000000000Z')
            #df.set_index('time',inplace=True)
            df_list.append(df)
    
    final = pd.concat(df_list)
    
    names = {
        'o': 'open',
        'c': 'close',
        'h': 'high',
        'l': 'low',
        0: 'vol',
        'time': 'time',
    }
    new_names = []
    for column_name in final.columns:
        new_names.append(names[column_name])
    final.columns = new_names
    
    return final

In [4]:
def cal(client, instrument, perdiod):

    
    
    params = {
        "instrument": instrument,
        "period": perdiod
    }
    
    # PERIOD VALUES
    #3600 - 1 hour
    #43200 - 12 hours
    #86400 - 1 day
    #604800 - 1 week
    #2592000 - 1 month
    #7776000 - 3 months
    #15552000 - 6 months
    #31536000 - 1 year
    # http://developer.oanda.com/rest-live/forex-labs/

    r = labs.Calendar(params=params)
    client.request(r)
    
    df = pd.DataFrame.from_dict(r.response, orient='columns')
    
    df['timestamp'] = pd.to_datetime(df['timestamp']*1000000000)
    df = df[['impact', 'timestamp']]
    df.columns = ['impact', 'time']

    return df.groupby('time').sum().reset_index()

In [5]:
def merge(history, calendar):
    return pd.merge(history, calendar, left_on = 'time', right_on = 'time', how='outer')\
                                                                            .set_index('time')\
                                                                            .astype(float)\
                                                                            .fillna(0)

In [6]:
def broaden_impact(df, period):
    df = df.reset_index().sort_values('time').set_index('time')

    for i in range(periods):
        df.loc[
            (df['impact'].shift(-1-i) != 0)
        ,'impact'] = df['impact'].shift(-1-i)
    
    df['impact'] = df['impact'].fillna(0)

    return df[df['low'] != 0]

In [7]:
client = API(access_token='f8599fa0624567b98d6293acc87489bb-e288ec05b46b6e3d0bc753e6a2fbab48')
aid = '101-004-8182547-007'

In [8]:
granularity_param = 'M5'

In [9]:
calendar = cal(client, 'EUR_USD', 604800)

In [10]:
history = hist(client, 'EUR_USD', 2, 0, granularity_param)

In [11]:
merged = merge(history, calendar)

In [12]:
merged = merged[merged['high'] != 0]

## Use params to modify sample data

In [13]:
averageMultiplayer = 1.3
bottomBarrierPips = 0.0040
endingHour = 12
openingInterval = 1
slPips = 0.0015
startingHour = 8
tpMultiplier = 1.4

tradeUnits = 100
instrument = 'EUR_USD'

In [14]:
# TO BE REMOVED
OpenSample = merged[
    (
        (merged.index.hour >= 2) & 
        (merged.index.hour < startingHour + openingInterval) &
        (merged.index.minute.isin([20, 40]))
    ) |
    ( 
        (merged.index.hour == startingHour + openingInterval) &
        (merged.index.minute.isin([0]))
    )
]

In [15]:
OpenSample['ma5'] = OpenSample['close'].rolling(5).mean()
OpenSample['ma10'] = OpenSample['close'].rolling(10).mean()
OpenSample = OpenSample.dropna()

/home/jacek/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
/home/jacek/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [16]:
OpenSample

,open,high,low,close,vol,impact,ma5,ma10
time,,,,,,,,
2019-04-08 06:40:00,1.12288,1.12299,1.12282,1.12286,41.0,0.0,1.122812,1.122725
2019-04-08 07:20:00,1.12255,1.12270,1.12240,1.12247,58.0,0.0,1.122702,1.122755
2019-04-08 07:40:00,1.12220,1.12238,1.12218,1.12230,43.0,0.0,1.122566,1.122743
2019-04-08 08:20:00,1.12288,1.12322,1.12285,1.12308,57.0,0.0,1.122664,1.122775
2019-04-08 08:40:00,1.12368,1.12373,1.12326,1.12329,39.0,0.0,1.122800,1.122812
2019-04-08 09:00:00,1.12337,1.12353,1.12328,1.12348,27.0,0.0,1.122924,1.122868
2019-04-09 02:20:00,1.12649,1.12650,1.12647,1.12650,15.0,0.0,1.123730,1.123216
2019-04-09 02:40:00,1.12669,1.12673,1.12658,1.12660,12.0,0.0,1.124590,1.123578
2019-04-09 03:20:00,1.12655,1.12655,1.12650,1.12650,3.0,0.0,1.125274,1.123969


In [17]:
OpenSample = OpenSample.iloc[:-1]

## Opening function

So, each 20 minutes there will be crone job scheduled with this python script. Every 20 minutes it will check which candle/hour it is, and depending on it will do some action

In [18]:
# openTrade(client, aid, 'EUR_USD', 100, 0.0020, 0.0040)

In [19]:
# len(get_trades(client, aid)['trades'])

In [20]:
# EACH TIME OPEN PRICE OF CANDLE (CURRENT PRICE ACTUALY) WILL MATTER
tradeUnits = 100
instrument = 'EUR_USD'
candleData = OpenSample

def openTrade(api, aid, instrument, tradeUnits, sl, tp):
    data = {
        "order":{
            "instrument": instrument,
            "units": tradeUnits,
            "side": "sell",
            "type":"MARKET",
            "stopLossOnFill": {
              "timeInForce": "GTC",
              "price": str(sl),
            },
            "takeProfitOnFill": {
              "timeInForce": "GTC",
              "price": str(tp),
            },
            },
            "lowerBound": 0.0005,
            "upperBound": 0.0005,   
        }

    o = orders.OrderCreate(aid, data=data)
    return api.request(o)


def get_trades(api, aid):
    trades_list = trades.OpenTrades(accountID=aid)
    return api.request(trades_list)['trades']

def change_sl_tp(client, aid, tradeID, sl, tp):
    data = {
      "takeProfit": {
        "timeInForce": "GTC",
        "price": str(tp)
      },
      "stopLoss": {
        "timeInForce": "GTC",
        "price": str(sl)
      }
    }

    r = trades.TradeCRCDO(accountID=aid,
                           tradeID=tradeID,
                           data=data)
    return client.request(r)

if len(candleData[candleData['impact'] != 0]) == 0:
    currentCandle = candleData.iloc[-1]
    
    # FIRST OPEN
    if (currentCandle.name.hour == startingHour and\
        currentCandle.name.minute == 20):
        
        openTrade(client, aid, instrument, tradeUnits, bottomBarrierPips, 0.0100)
    
    # OPEN OTHERS
    if ((currentCandle.name.hour == startingHour) and\
        (currentCandle.name.minute == 40)) or\
       ((currentCandle.name.hour > startingHour) and\
        (currentCandle.name.hour < startingHour + openingInterval)):
        
        if currentCandle['ma5'] > currentCandle['ma10']:
        
            tradesList = get_trades(client, aid)
            if len(tradesList) != 0:
                firstTimestamp = '2050-10-28T14:28:05.231759081Z'
                pricesContainer = []
                for trade in tradesList:
                    pricesContainer.append(trade['price'])
                    if trade['openTime'] < firstTimestamp:
                        firstTimestamp = trade['openTime']
                        firstSL = trade['stopLossOnFill']['price']
                        basePrice = trade['price']
                    # EDGE CASE: trades before were closed because of bottomBarrier
                    # (or stoploss in other words), so it will open another trade
                    if currentCandle['open'] > firstSL:
                        if currentCandle['open'] < basePrice:
                            if currentCandle['open'] < min(pricesContainer):
                                unitsMultiplayed = len(pricesContainer) * averageMultiplayer
                                openTrade(client, aid, instrument, unitsMultiplayed, firstSL, 0.0100)

        print('next candles')
        
    # OPEN THE LAST ONE 
    # MOVE SL AT THE END OF OPENING INTERVAL
    elif (currentCandle.name.hour == startingHour + openingInterval and\
        currentCandle.name.minute == 0):
        
        if currentCandle['ma5'] > currentCandle['ma10']:
        
            tradesList = get_trades(client, aid)
            if len(tradesList) != 0:
                firstTimestamp = '2050-10-28T14:28:05.231759081Z'
                pricesContainer = []
                for trade in tradesList:
                    pricesContainer.append(trade['price'])
                    unitsContainer.append(trade['initialUnits'])
                    if trade['openTime'] < firstTimestamp:
                        firstTimestamp = trade['openTime']
                        firstSL = trade['stopLossOnFill']['price']
                        basePrice = trade['price']
                    # EDGE CASE: trades before were closed because of bottomBarrier
                    # (or stoploss in other words), so it will open another trade
                    if currentCandle['open'] > firstSL:
                        if currentCandle['open'] < basePrice:
                            if currentCandle['open'] < min(pricesContainer):
                                unitsMultiplayed = len(pricesContainer) * averageMultiplayer
                                openTrade(client, aid, instrument, unitsMultiplayed, firstSL, 0.0100)
        
        pricesContainer = []
        unitsContainer = []
        for trade in get_trades(client, aid):
            pricesContainer.append(trade['price'])
            unitsContainer.append(trade['initialUnits'])
        
         if len(pricesContainer) != 0:
            weightedPrices = []
            for i in range(len(pricesContainer)):
                weightedPrices.append(pricesContainer[i] * unitsContainer[i])
            averagePrice = weightedPrices / sum(unitsContainer)
            newSL = averagePrice - slPips
            newTP = averagePrice + (slPips * tpMultiplier)

            for trade in get_trades(client, aid):
                change_sl_tp(client, aid, trade['id'], newSL, newTP)
    
        # EDGE CASE: IT CAN TRY TO MOVE SL TOO CLOSE TO TRADES
        print('last one')
        
    
    # CLOSE TRADES (ENDING BARRIER)
    elif currentCandle.name.hour == endingHour:
        
        for trade in get_trades(client, aid):
            trades.TradeClose(accountID=aid, data={"units": trade['initialUnits']})
        
        print('ending hour')
    
    else:
        print('no action')

IndentationError: unindent does not match any outer indentation level (<ipython-input-20-79cc4d9269f4>, line 115)